In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import numpy as np

In [2]:
# Get configuration file that has all the api tokens
def configure():
    load_dotenv()

In [3]:
#Grab the authorization and client token from the env
configure()
UNDOC_AUTHORIZATION = (os.getenv('UNDOC_AUTHORIZATION'))
UNDOC_CLIENT_TOKEN = (os.getenv('UNDOC_CLIENT_TOKEN'))

***

## Get the play counts of each song in the Album

#### `request_album_playcounts()`

In [4]:
def request_album_playcounts(album_ID):
    '''Search ID of albums and return response to get playcounts and track_uri'''
    url = "https://api-partner.spotify.com/pathfinder/v1/query"

    querystring = {
                "operationName":"queryAlbumTracks",
                "variables":'{\"uri\":\"spotify:album:' +album_ID+ '\",\"offset\":0,\"limit\":300}',
                "extensions":"{\"persistedQuery\":{\"version\":1,\"sha256Hash\":\"f387592b8a1d259b833237a51ed9b23d7d8ac83da78c6f4be3e6a08edef83d5b\"}}"
                }

    headers = {
    "authority": "api-partner.spotify.com",
    "accept": "application/json",
    "accept-language": "en",
    "app-platform": "WebPlayer",
    "authorization":"Bearer {}".format(UNDOC_AUTHORIZATION),
    "client-token": str(UNDOC_CLIENT_TOKEN),
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://open.spotify.com",
    "referer": "https://open.spotify.com/"
}

    #prcoessing the result
    result = requests.request("GET", url, headers=headers, params=querystring)
    json_result = result.json()
    #getting each track object
    items = json_result['data']['albumUnion']['tracks']['items']
    
    # get the number of items to determine the loop size
    n = len(items)
    track_uri_list = np.empty(n, dtype=object)
    playcount_list = np.empty(n)
    for i in range(n):
        track_uri_list[i] = items[i]['track']['uri']
        playcount_list[i] = items[i]['track']['playcount']

    return track_uri_list, playcount_list

Test what `request_album_playcounts()` return

In [5]:
request_album_playcounts('5MS3MvWHJ3lOZPLiMxzOU6')

(array(['spotify:track:1bDbXMyjaUIooNwFE9wn0N',
        'spotify:track:46s57QULU02Voy0Kup6UEb',
        'spotify:track:34tz0eDhGuFErIuW3q4mPX',
        'spotify:track:0wshkEEcJUQU33RSRBb5dv',
        'spotify:track:7l2nxyx7IkBX5orhkALg0V',
        'spotify:track:2ZL7WZcjuYKi1KUDtp4kCC',
        'spotify:track:0sSRLXxknVTQDStgU1NqpY',
        'spotify:track:4Flfb4fGscN9kXPOduQLrv',
        'spotify:track:7GeTsDIc5ykNB6lORO6Cee',
        'spotify:track:2KLwPaRDOB87XOYAT2fgxh',
        'spotify:track:45pUWUeEWGoSdH6UF162G8',
        'spotify:track:410TZrK18uRjtsTunG14cl',
        'spotify:track:2ZlABhxMMa43COmZ97kKsJ',
        'spotify:track:29YW2xXlo6Pc6M0SBTCXYN',
        'spotify:track:1MTXYvJ9TSqg9x6WPCDx2n',
        'spotify:track:31bsuKDOzFGzBAoXxtnAJm'], dtype=object),
 array([3.77513209e+08, 1.19683551e+08, 1.15641417e+08, 7.09288030e+07,
        7.98015990e+07, 1.04130454e+08, 6.05025210e+07, 4.76703610e+07,
        1.00836124e+08, 1.38410520e+08, 7.70638210e+07, 4.49430490e+07,


#### `get_today_playcounts()`

In [6]:
def get_today_playcounts(album_ID):
    '''Using the undocumented API to request playcounts and return a dataframe of '''
    track_uri_list, playcount_list = request_album_playcounts(album_ID)
    df = pd.DataFrame({'track_uri':track_uri_list, 'playcount':playcount_list})
    df['date_tracked'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    return df

In [7]:
get_today_playcounts("5MS3MvWHJ3lOZPLiMxzOU6")

,track_uri,playcount,date_tracked
0,spotify:track:1bDbXMyjaUIooNwFE9wn0N,377513209.0,2023-03-02
1,spotify:track:46s57QULU02Voy0Kup6UEb,119683551.0,2023-03-02
2,spotify:track:34tz0eDhGuFErIuW3q4mPX,115641417.0,2023-03-02
3,spotify:track:0wshkEEcJUQU33RSRBb5dv,70928803.0,2023-03-02
4,spotify:track:7l2nxyx7IkBX5orhkALg0V,79801599.0,2023-03-02
5,spotify:track:2ZL7WZcjuYKi1KUDtp4kCC,104130454.0,2023-03-02
6,spotify:track:0sSRLXxknVTQDStgU1NqpY,60502521.0,2023-03-02
7,spotify:track:4Flfb4fGscN9kXPOduQLrv,47670361.0,2023-03-02
8,spotify:track:7GeTsDIc5ykNB6lORO6Cee,100836124.0,2023-03-02
9,spotify:track:2KLwPaRDOB87XOYAT2fgxh,138410520.0,2023-03-02
